### Data generation on IBMQ Casablanca

In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import numpy
import copy
import qiskit
from qiskit.test.mock import FakeCasablanca
from mitiq import zne
import mitiq
from qiskit.providers.aer.extensions import SnapshotDensityMatrix
from mitiq.zne.scaling import fold_gates_from_left,fold_gates_at_random, fold_global,fold_gates_from_right
from mitiq.zne.inference import LinearFactory,RichardsonFactory,PolyFactory
import qiskit.providers.aer.noise as noise
import ipyparallel as ipp
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit.providers.aer import AerSimulator
from openfermion.linalg import givens_decomposition_square as givensRotns
from qiskit.opflow import I,X,Y,Z,StateFn,AbelianGrouper
from qiskit.opflow.primitive_ops import PauliSumOp
from qiskit_nature.mappers.second_quantization import JordanWignerMapper,ParityMapper
from qiskit_nature.problems.second_quantization.electronic.builders import fermionic_op_builder
from qiskit.quantum_info import Pauli
from qiskit.chemistry import FermionicOperator
from qiskit.aqua.operators.legacy import op_converter
from openfermion.circuits import slater_determinant_preparation_circuit
from qiskit import QuantumCircuit,execute,QuantumRegister,ClassicalRegister
from qiskit import Aer
from qiskit.circuit.random import random_circuit
from joblib import Parallel,delayed
from qiskit.aqua.operators import WeightedPauliOperator,Z2Symmetries
from qiskit.chemistry.components.variational_forms import UCCSD
import scipy
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

In [2]:
backend = FakeCasablanca() #Aer.get_backend('qasm_simulator')
def egBandHamiltonianPartition(U):
    #Getting chemical Potential for Half-Filling
    with open('chem_pot_for_Half_Fill.txt','r') as f:
        lines=f.readlines()[1:]
        for line in lines:
            elems=line.split()
            if int(elems[0])==U:
                muHalf=float(elems[1]) #Chem Pot for a given Hubbard U
    #Getting the one body and two body interaction vertexes
    with open('v1e.dat','r') as f:
            lines=f.readlines()[1:]
            num_sites=4
            chem_pot=numpy.zeros((2*num_sites,2*num_sites))
            eg_h1_0=numpy.zeros((2*num_sites,2*num_sites))
            eg_h1_1=numpy.zeros((2*num_sites,2*num_sites))
            for line in lines:
                line=line.split()
                i,j=map(int,line[:2])
                val=float(line[2])
                if(i==j):
                    eg_h1_0[i,j]=eg_h1_0[i+num_sites,j+num_sites]=val
                else:
                    eg_h1_1[i,j]=eg_h1_1[i+num_sites,j+num_sites]=val
            for i in range(2*num_sites):
                chem_pot[i][i]=-muHalf
            eg_h1_0=eg_h1_0+chem_pot       
    with open('v2e.dat','r') as f:
        num_sites=4
        eg_h2_0=numpy.zeros((2*num_sites,2*num_sites,2*num_sites,2*num_sites))
        eg_h2_1=numpy.zeros((2*num_sites,2*num_sites,2*num_sites,2*num_sites))
        for line in f:
            if "#" in line:
                continue
            line = line.split()
            i,j,k,l = map(int, line[:4])
            val = float(line[4])
            if((i==j) and (k==l)):
                eg_h2_0[i,j,k,l] = eg_h2_0[i+num_sites,j+num_sites,k,l] = eg_h2_0[i,j,k+num_sites,l+num_sites] \
                = eg_h2_0[i+num_sites,j+num_sites,k+num_sites,l+num_sites] = 0.5*val  # convention with 0.5 factor included.
            else:
                eg_h2_0[i,j,k,l] = eg_h2_0[i+num_sites,j+num_sites,k+num_sites,l+num_sites] = 0.5*val  # convention with 0.5 factor included.
                eg_h2_1[i+num_sites,j+num_sites,k,l] = eg_h2_1[i,j,k+num_sites,l+num_sites] = 0.5*val
    return eg_h1_0,eg_h1_1,eg_h2_0,eg_h2_1
def Energy(params):
    circ=var_form_base.construct_circuit(parameters=params)
    state=execute(circ,Aer.get_backend('statevector_simulator'),shots=1024).result().get_statevector()
    E=(state.conj()@Hmat@state).real
    return E
def qubitOp(h1,h2):  
    fer_op=fermionic_op_builder.build_ferm_op_from_ints(h1,h2)
    qubit_conv = QubitConverter(ParityMapper(), two_qubit_reduction=True, z2symmetry_reduction="auto")
    qubit_op = qubit_conv.convert(fer_op, len(h1)//2)
    #mapper=JordanWignerMapper()
    #qubit_op=mapper.map(fer_op)
    return qubit_op 
def LRF_2_body_OD_terms(h2): #For 2 body off-diagonal terms
    def reshape_vec_to_mat(eigs):
        weight,vec1,vec2=eigs
        num_qubits=int(numpy.sqrt(vec1.shape[0]))
        L1=numpy.reshape(vec1,(num_qubits,num_qubits))
        L2=numpy.reshape(vec2,(num_qubits,num_qubits))
        return [numpy.sqrt(numpy.complex(weight))*L1,numpy.sqrt(numpy.complex(weight))*L2]
    #checksum  
    def higherRankRep(L):
        T=numpy.zeros((num_qubits,num_qubits,num_qubits,num_qubits))*1j
        for p in range(num_qubits):
            for q in range(num_qubits):
                for r in range(num_qubits):
                    for s in range(num_qubits):
                        T[p,q,r,s]=L[p,q]*L[r,s]
                        
        return T
    def get_givens_rotns(L):
        num_qubits=len(L)
        qubitH=qubitOp(numpy.zeros((8,8)),h2_X)
        num_qubits_red=qubitH.num_qubits
        H=qubitH.to_matrix()
        givens=givensRotns(L)[0]
        rotH=qubitH
        #print(givens,rotH)
        Ops=['IIIIIY','IIYIII']
        circ=QuantumCircuit(num_qubits_red)
        for l in range(len(givens)):
            rots=givens[l]
            for tup in rots:
                i,j,theta,phi=tup
                h1=numpy.zeros((num_qubits,num_qubits))*1j
                h1[i,j]=-1j
                h1[j,i]=1j
                qubit_h1=qubitOp(h1,numpy.zeros((num_qubits,num_qubits,num_qubits,num_qubits)))
                Op=PauliSumOp.from_list([('I'*num_qubits_red,1)])
                List=[(Ops[l],0.5)]
                for i in range(len(List)):
                    Op=PauliSumOp.from_list([('I'*num_qubits_red,numpy.cos(theta*List[i][1])),(List[i][0],-1j*numpy.sin(theta*List[i][1]))])@Op
                    circ=add_multiqubit_gate(List[i][0], theta*List[i][1], circ)  
                rotH=Op@rotH@Op.adjoint()
                rotH=rotH.reduce().reduce()
        return circ,rotH,qubitH
    #constructing Low rank factorization circuits
    #Super matrix rep of four rank tensor (NxNxNxN)->(N^2xN^2)
    num_qubits=len(h2)
    N2=num_qubits*num_qubits
    h2_pq_rs=numpy.reshape(h2,(N2,N2))
    #Diagonalize Super matrix
    #u,d,v=numpy.linalg.svd(h2_pq_rs,full_matrices=True)
    w,v=numpy.linalg.eigh(h2_pq_rs)
    #Build Cholesky vectors
    Larr=[]
    arr=[((w[i],v[:,i],v.conj().T[i])) for i in range(len(v)) if numpy.abs(w[i])>1e-4]
    Larr=list(map(reshape_vec_to_mat,arr))
    qc,rotH,qubit_h2=get_givens_rotns(Larr[0][0])
    return qc,rotH,qubit_h2

def add_multiqubit_gate(pauli_string, param, circuit):
    num_qubits=circuit.num_qubits
    qr=QuantumRegister(num_qubits,'q')
    if pauli_string == 'I'*num_qubits:
        gate = 1
        for j in range(len(pauli_string)):
            gate = numpy.kron(gate, Pauli('I').to_matrix())
        gate *= -1j * np.sin(param)
        gate += numpy.cos(param) * np.eye(2**num_qubits)
        circuit.unitary(gate, qr, label=pauli_string)
    else:
        qubits_to_act_on = []
        gate = 1
        for j in range(len(pauli_string)):
            if pauli_string[j] != 'I':
                gate=numpy.kron(Pauli(pauli_string[j]).to_matrix(),gate)
                qubits_to_act_on.append(num_qubits-j-1)
        gate *= (-1j * numpy.sin(param))
        gate += numpy.cos(param) * numpy.eye(2**len(qubits_to_act_on))
        List_regs=[qr[i] for i in qubits_to_act_on]#[::-1]
        Label=pauli_string+"\n{:0.02f}".format(param.real)
        circuit.unitary(gate, List_regs, label = Label)
    return circuit  

def get_givens_rotns(h):
        num_qubits=len(h)
        qubitH=qubitOp(h,numpy.zeros((num_qubits,num_qubits,num_qubits,num_qubits)))
        num_qubits_red=qubitH.num_qubits
        H=qubitH.to_matrix()
        w,v=numpy.linalg.eigh(h)
        givens=givensRotns(v.T)[0]
        rotH=qubitH
        circ=QuantumCircuit(num_qubits_red)
        for rots in givens:
            for tup in rots:
                i,j,theta,phi=tup
                h1=numpy.zeros((num_qubits,num_qubits))*1j
                h1[i,j]=-1j
                h1[j,i]=1j
                qubit_h1=qubitOp(h1,numpy.zeros((num_qubits,num_qubits,num_qubits,num_qubits)))
                List=qubit_h1.primitive.to_list()
                Op=PauliSumOp.from_list([('I'*num_qubits_red,1)])
                for i in range(len(List)):
                    Op=PauliSumOp.from_list([('I'*num_qubits_red,numpy.cos(theta*List[i][1])),(List[i][0],1j*numpy.sin(theta*List[i][1]))])@Op
                    circ=add_multiqubit_gate(List[i][0], theta*List[i][1], circ)
                rotH=Op@rotH@Op.adjoint()
                rotH=rotH.reduce().reduce()
        return circ,rotH,qubitH
    
def doubly_decomposed_form(h1_D,h1_X,h2_D,h2_X):

    #getting the givens rotation circuit and the rotated Hamiltonian for the one body terms
    circ_1_body,rotH_1_X,qubitH_1_X=get_givens_rotns(h1_X)
    #getting Cholesky vectors for two body off-diag terms
    circ_2_body,rotH_2_X,qubitH_2_X=LRF_2_body_OD_terms(h2_X)
    qubitH_D=qubitOp(h1_D,h2_D)
    circs_and_rotH_arr=[(circ_1_body,rotH_1_X,qubitH_1_X),(circ_2_body,rotH_2_X,qubitH_2_X),(qubitH_D)]
    return circs_and_rotH_arr

def Energy(params):
    circ=var_form_base.construct_circuit(parameters=params)
    state=numpy.array(list(execute(circ,Aer.get_backend('statevector_simulator'),shots=1024).result().get_statevector()))
    E=(state.conj()@Hmat@state).real
    return E

# def hamiltonian_estimation_in_doubly_decomposed_form_qasm(params,num_shots_arr=[2**18,2**18,2**18],measure_with_noise=False):
#     def Energy_Estimator_DD(input_vars):
#         def getCounts(Input,noise_model=None,basis_gates=None):
#             circ,num_shots=Input
#             circ1=circ.copy()
#             circ1.add_register(ClassicalRegister(6,'c'))
#             circ1.measure([0,1,2,3,4,5],[0,1,2,3,4,5]) 
#             job = execute(circ1, backend,shots=num_shots)#,basis_gates=basis_gates,noise_model=noise_model,had_transpiled=True) 
#             res = job.result()
#             counts = res.get_counts(circ1) 
#             return counts
#         def estimate(PauliOp,counts):
#             PauliString,coeff=PauliOp.primitive.to_list()[0]
#             indexOfZs=numpy.where(numpy.array(list(PauliString))=='Z')[0]
#             bitstrings,cnts=list(counts. keys()),list(counts.values())
#             p=0
#             count_valid=0
#             for i in range(len(bitstrings)):
#                 bit_Str_to_Arr=numpy.array(list(bitstrings[i]))
#                 #flag1=1 if len(numpy.where(bit_Str_to_Arr=='1')[0])==4 else 0 #check number of electrons
#                 #flag2=1 if len(numpy.where(bit_Str_to_Arr[0:4]=='1')[0])==2 else 0 #check net spin
#                 #if ((flag1==1) and (flag2==1)):
#                 #count_valid=count_valid+cnts[i]
#                 counter_1=list(numpy.array(list(bitstrings[i]))[indexOfZs]).count('1')
#                 if counter_1%2==1:
#                     p=p+cnts[i]
#             p=p/sum(cnts)
#             mean=(1-2*p)
#             return mean*coeff.real
#         def meanOp(PauliOp,counts):
#             mean=0
#             for i in range(len(PauliOp)):
#                 mean=mean+estimate(PauliOp[i],counts)
#             return mean 
#         def covariance(Op,i,j,counts):
#             cov=estimate(Op[i]@Op[j],counts)-estimate(Op[i],counts)*estimate(Op[j],counts)
#             return cov
#         def covbtnOps(A,B,counts):
#             corr=0
#             for i in range(len(A)):
#                 for j in range(len(B)):
#                     corr=corr+estimate(A[i]@B[j],counts)-estimate(A[i],counts)*estimate(B[j],counts)
#             return corr        
#         def variance(Op,counts):
#             var=0
#             cov_mat=numpy.zeros((len(Op),len(Op)))
#             for i in range(len(Op)):
#                 for j in range(i,len(Op)):
#                     if i==j:
#                         mean=estimate(Op[i],counts)
#                         coeff=Op[i].primitive.to_list()[0][1].real
#                         cov_mat[i][j]=(coeff**2-mean**2)
#                         var=var+(coeff**2-mean**2)
#                     elif i!=j:
#                         var=var+2*covariance(Op,i,j,counts)
#                         cov_mat[i][j]=cov_mat[j][i]=covariance(Op,i,j,counts)
#             return var
#         def OpErr(data):
#             Op,counts=data
#             var=variance(Op,counts)
#             err=numpy.sqrt(var/sum(counts.values()))
#             return err.real
#         if len(input_vars)==5:
#             circ,Op,num_shots,noise_model,basis_gates=input_vars
#             counts_Arr=getCounts((circ,num_shots),noise_model,basis_gates)
#             m1=meanOp(Op,counts_Arr)
#             err1=OpErr((Op,counts_Arr))
#         else:
#             circ,Op,num_shots=input_vars
#             counts_Arr=getCounts((circ,num_shots))
#             m1=meanOp(Op,counts_Arr)
#             err1=OpErr((Op,counts_Arr))
#         return m1,err1,counts_Arr
#     #params,num_shots_arr=input_vars
#     ansatz_circ=var_form_base.construct_circuit(params)
#     circ_rot_Arr=[ansatz_circ.copy(),ansatz_circ.copy(),ansatz_circ.copy()]
#     circ_rot_Arr[0]=circ_rot_Arr[0]+circs_and_rotH[0][0]
#     circ_rot_Arr[1]=circ_rot_Arr[1]+circs_and_rotH[1][0]
#     #Estimate from statistics
#     data_arr=[(circ_rot_Arr[0],circs_and_rotH[0][1],num_shots_arr[0]),
#               (circ_rot_Arr[1],circs_and_rotH[1][1],num_shots_arr[1]),
#               (circ_rot_Arr[2],circs_and_rotH[2],num_shots_arr[2])]
#     results=Parallel(n_jobs=1,verbose=2)(delayed(Energy_Estimator_DD)(data_arr[i]) for i in range(len(data_arr)))
#     m1,err1,counts1=results[0][0],results[0][1],results[0][2]
#     m2,err2,counts2=results[1][0],results[1][1],results[1][2]
#     m3,err3,counts3=results[2][0],results[2][1],results[2][2]
#     return m1+m2+m3,err1+err2+err3,[counts1,counts2,counts3]
# def countYgates(pauli_label):
#     countYgates = sum(map(lambda x : 1 if 'Y' in x else 0, pauli_label))
#     return countYgates

In [3]:
U=7
h1_D,h1_X,h2_D,h2_X=egBandHamiltonianPartition(U)
circs_and_rotH=doubly_decomposed_form(h1_D,h1_X,h2_D,h2_X)    
def qubitWeightedOp(h1,h2):
    qubit_op=FermionicOperator(h1,h2).mapping('parity')
    qubitH=Z2Symmetries.two_qubit_reduction(qubit_op, qubit_op.num_qubits//2)
    return qubitH
qubitH=qubitWeightedOp(h1_D+h1_X,h2_D+h2_X)
Hmat=op_converter.to_matrix_operator(qubitH).dense_matrix
w,v=numpy.linalg.eigh(Hmat)
Eg=w[0]
circ=QuantumCircuit(6)
circ.x(0)
circ.x(3)
var_form_base=UCCSD(8,num_particles=4, initial_state=circ,qubit_mapping='parity',two_qubit_reduction=True)
var_form_base.manage_hopping_operators()

In [4]:
with open('OptStepsWithqasm_parityU=7Fin.txt','r') as f:
    lines=f.readlines()
    Labels=[]
    for i in range(len(lines)):
        if lines[i][0]=='l':
            Labels.append(lines[i].split('-')[1][1:-1])
    params=eval(lines[-1])    
for i in range(len(Labels)):
    var_form_base.push_hopping_operator(WeightedPauliOperator([[1j,Pauli(Labels[i])]]))   

In [5]:
#Number operator
h_N=numpy.eye(8)
qubitN=qubitOp(h_N,numpy.zeros((8,8,8,8)))
N_matrix=qubitN.to_matrix()
#Spin Sz operator
h_Sz=numpy.block([[numpy.eye(4),numpy.zeros((4,4))],
                  [numpy.zeros((4,4)),-1*numpy.eye(4)]])
qubitSz=qubitOp(h_Sz,numpy.zeros((8,8,8,8)))
Sz_matrix=qubitSz.to_matrix()

In [6]:
#Function checks number of particles corresponding to a bitstring config
def NumParticlesBitstring(bit_str):
    state_vector=numpy.zeros(2**6)
    state_vector[int(bit_str,2)]=1
    return numpy.real(state_vector@N_matrix@state_vector.T)
#Function checks Spin Sz corresponding to a bitstring config
def SpinBitstring(bit_str):
    state_vector=numpy.zeros(2**6)
    state_vector[int(bit_str,2)]=1
    return numpy.real(state_vector@Sz_matrix@state_vector.T)
#Subspace filtering func based on expectation
def filter_bitstrings(counts):
    Keys=list(counts.keys())
    Vals=list(counts.values())
    newCounts={}
    for i in range(len(Keys)):
        n=NumParticlesBitstring(Keys[i])
        s=SpinBitstring(Keys[i])
        if (numpy.abs(n-4)<1e-3 and  numpy.abs(s)<1e-3):
            newCounts[Keys[i]]=Vals[i]
        else:
            continue
    return newCounts        

In [7]:
#Constructing Confusion matrix for IBM Q device
def confusion_matrix(num_qubits,noisy_backend,num_shots):
    num_qubits=6
    Dict_bitstr={}
    for i in range(2**num_qubits):
        qc=QuantumCircuit(num_qubits)
        bit_str=str(format(i,'b'))
        pos_Xs=numpy.where(numpy.array(list(bit_str)[::-1])=='1')[0]
        for ind in pos_Xs:
            qc.x(ind)
        qc=qiskit.transpile(qc,backend=noisy_backend,initial_layout=[0,1,2,3,4,5])
        qc.add_register(ClassicalRegister(num_qubits,'c'))
        qc.measure([0,1,2,3,4,5],[0,1,2,3,4,5]) 
        job = noisy_backend.run(qc, shots=num_shots)
        #job = execute(qc, noisy_backend,shots=num_shots)#,basis_gates=basis_gates,noise_model=noise_model,had_transpiled=True) 
        res = job.result()
        counts = res.get_counts(qc)     
        Dict_bitstr[bit_str]=counts
    M=numpy.zeros((2**num_qubits,2**num_qubits))
    for i in range(2**num_qubits):
        transfer_map=Dict_bitstr[str(format(i,'b'))]
        Keys=list(transfer_map.keys())
        Values=list(transfer_map.values())
        for j in range(len(Keys)):
            print(i,int(Keys[j],2),Values[j]/num_shots)
            M[i][int(Keys[j],2)]=Values[j]/num_shots
    return M
def measurement_mitigate(M,counts):
    vec=numpy.zeros(M.shape[0])
    Keys=list(counts.keys())
    num_shots=sum(list(counts.values()))
    for i in range(len(Keys)):
        vec[int(Keys[i],2)]=counts[Keys[i]]/num_shots
        
    actual_vec=numpy.linalg.inv(M)@vec
    new_counts={}
    for i in range(len(Keys)):
        new_counts[Keys[i]]=actual_vec[int(Keys[i],2)]  
    return new_counts   

In [8]:
noisy_backend = FakeCasablanca() 
basis_gates=noisy_backend.configuration().basis_gates

In [9]:
#numpy.set_printoptions(precision=3,suppress=True)
#noisy_backend=FakeCasablanca()
#M=confusion_matrix(6,noisy_backend,2**14)
M=numpy.load('confusion_matrix_IBMQ_Casablanca.npy',allow_pickle=True)


In [40]:
#The Hamiltonian is divided into three mutually commuting parts each can be rotated to a diagonal representation 
#by appending additional gates that correspond to single body rotations.
#function that measures first part of Hamiltonian
def executor1(circuit, shots=2**14,useSubspaceFilter=True,measure_mitiq=True):
    """Executes the input circuit and returns the noisy expectation value <A>, where A=|00>00|.
    """
    # Select a noisy backend
    # noisy_backend = qiskit.IBMQ.load_account().get_backend("ibmq_lima")
    noisy_backend = FakeCasablanca() # Simulator with noise model similar to "ibmq_lima"

    # Append measurements
    circuit_to_run = circuit.copy()
    circuit_to_run=qiskit.transpile(circuit_to_run,backend=noisy_backend,initial_layout=initial_layout1,optimization_level=0)
    circuit_to_run.add_register(ClassicalRegister(6,'c'))
    circuit_to_run.measure([0,1,2,3,4,5],[0,1,2,3,4,5])
    # Run and get counts
    print(f"Executing circuit with {len(circuit_to_run),circuit_to_run.count_ops()['cx']} gates using {shots} shots.")
    #qasm_str=circuit_to_run.qasm()
    #with open('circuit1.txt','+a') as f:
    #    print (qasm_str+'\n***************\n',file=f)
    job = noisy_backend.run(circuit_to_run, shots=shots)
    counts = job.result().get_counts()
    old_tot=1
    NumOld=len(counts.values())
    if (measure_mitiq==True):
        counts=measurement_mitigate(M,counts)
        old_tot=2**14
    #jsonString = json.dumps(counts)    
    #jsonFile = open("circuit1_counts_not_filtered.json", "a")
    #jsonFile.write(f"Executing circuit with {len(circuit_to_run),circuit_to_run.count_ops()['cx']} gates using {shots} shots.\n"+jsonString+'\n')
    #jsonFile.close()        
    if (useSubspaceFilter==True):
        counts=filter_bitstrings(counts)
    #jsonString = json.dumps(counts)    
    #jsonFile = open("circuit1_counts_filtered.json", "a")
    #jsonFile.write(f"Executing circuit with {len(circuit_to_run),circuit_to_run.count_ops()['cx']} gates using {shots} shots.\n"+jsonString+'\n')
    #jsonFile.close()    
    # Compute expectation value of the observable Op
    rho=numpy.zeros((2**6,2**6))
    Keys=list(counts.keys())
    Vals=list(counts.values())
    Num=len(Vals)
    tot=sum(Vals)
    #jsonFile = open("circuit1_counts_filtered.json", "a")
    #jsonFile.write(f"fraction of valid strings {Num/NumOld} fraction of valid measurements {tot*old_tot/shots}")
    #jsonFile.close()
    for i in range(len(Keys)):
        rho[int(Keys[i],2)][int(Keys[i],2)]=Vals[i]/tot
    noisy_value=numpy.real(numpy.sum(numpy.diag(rho@H_1.to_matrix())))
    print(noisy_value,'\n')
    return noisy_value
#function that measures second part of Hamiltonian
def executor2(circuit, shots=2**14, useSubspaceFilter=True,measure_mitiq=True):
    """Executes the input circuit and returns the noisy expectation value <A>, where A=|00>00|.
    """
    # Select a noisy backend
    # noisy_backend = qiskit.IBMQ.load_account().get_backend("ibmq_lima")
    noisy_backend = FakeCasablanca() 
    # Append measurements
    circuit_to_run=circuit.copy()
    circuit_to_run=qiskit.transpile(circuit_to_run,noisy_backend,initial_layout=initial_layout2,optimization_level=0)
    circuit_to_run.add_register(ClassicalRegister(6,'c'))
    circuit_to_run.measure([0,1,2,3,4,5],[0,1,2,3,4,5])
    # Run and get counts
    print(f"Executing circuit with {len(circuit_to_run),circuit_to_run.count_ops()['cx']} gates using {shots} shots.")
    #qasm_str=circuit_to_run.qasm()
    #with open('circuit2.txt','+a') as f:
    #    print (qasm_str+'\n***************\n',file=f)
    job = noisy_backend.run(circuit_to_run, shots=shots)
    counts = job.result().get_counts()
    old_tot=1
    NumOld=len(counts.values())
    if (measure_mitiq==True):
        counts=measurement_mitigate(M,counts)  
        old_tot=2**14
    #jsonString = json.dumps(counts)    
    #jsonFile = open("circuit2_counts_not_filtered.json", "a")
    #jsonFile.write(f"Executing circuit with {len(circuit_to_run),circuit_to_run.count_ops()['cx']} gates using {shots} shots.\n"+jsonString+'\n')
    #jsonFile.close()     
    if (useSubspaceFilter==True):
        counts=filter_bitstrings(counts)
    #jsonString = json.dumps(counts)    
    #jsonFile = open("circuit2_counts_filtered.json", "a")
    #jsonFile.write(f"Executing circuit with {len(circuit_to_run),circuit_to_run.count_ops()['cx']} gates using {shots} shots.\n"+jsonString+'\n')
    #jsonFile.close()        
    # Compute expectation value of the observable Op
    rho=numpy.zeros((2**6,2**6))
    Keys=list(counts.keys())
    Vals=list(counts.values())
    Num=len(Vals)
    tot=sum(Vals)
    #jsonFile = open("circuit2_counts_filtered.json", "a")
    #jsonFile.write(f"fraction of valid strings {Num/NumOld} fraction of valid measurements {tot*old_tot/shots}")
    #jsonFile.close()
    for i in range(len(Keys)):
        rho[int(Keys[i],2)][int(Keys[i],2)]=Vals[i]/tot
    noisy_value=numpy.real(numpy.sum(numpy.diag(rho@H_2.to_matrix())))
    print(noisy_value,'\n')
    return noisy_value
#function that measures second part of Hamiltonian
def executor3(circuit, shots=2**14, useSubspaceFilter=True,measure_mitiq=True):
    """Executes the input circuit and returns the noisy expectation value <A>, where A=|00>00|.
    """
    # Select a noisy backend
    # noisy_backend = qiskit.IBMQ.load_account().get_backend("ibmq_lima")
    noisy_backend = FakeCasablanca() 
    # Append measurements
    circuit_to_run = circuit.copy()
    circuit_to_run=qiskit.transpile(circuit_to_run,backend=noisy_backend,initial_layout=initial_layout3,optimization_level=0)
    circuit_to_run.add_register(ClassicalRegister(6,'c'))
    circuit_to_run.measure([0,1,2,3,4,5],[0,1,2,3,4,5])
    # Run and get counts
    print(f"Executing circuit with {len(circuit_to_run),circuit_to_run.count_ops()['cx']} gates using {shots} shots.")
    #qasm_str=circuit_to_run.qasm()
    #with open('circuit3.txt','+a') as f:
    #    print (qasm_str,'\n***************\n',file=f)
    job = noisy_backend.run(circuit_to_run, shots=shots)
    counts = job.result().get_counts()
    NumOld=len(counts.values())
    old_tot=1
    if (measure_mitiq==True):
        counts=measurement_mitigate(M,counts)
        old_tot=2**14
    #jsonString = json.dumps(counts)    
    #jsonFile = open("circuit3_counts_not_filtered.json", "a")
    #jsonFile.write(f"Executing circuit with {len(circuit_to_run),circuit_to_run.count_ops()['cx']} gates using {shots} shots.\n"+jsonString+'\n')
    #jsonFile.close()     
    if (useSubspaceFilter==True):
        counts=filter_bitstrings(counts)
    #jsonString = json.dumps(counts)
    #jsonFile = open("circuit3_counts_filtered.json", "a")
    #jsonFile.write(f"Executing circuit with {len(circuit_to_run),circuit_to_run.count_ops()['cx']} gates using {shots} shots.\n"+jsonString+'\n')
    #jsonFile.close()        
    # Compute expectation value of the observable Op
    rho=numpy.zeros((2**6,2**6))
    Keys=list(counts.keys())
    Vals=list(counts.values())
    Num=len(Vals)
    tot=sum(Vals)
    #jsonFile = open("circuit3_counts_filtered.json", "a")
    #jsonFile.write(f"fraction of valid strings {Num/NumOld} fraction of valid measurements {tot*old_tot/shots}")
    #jsonFile.close()
    for i in range(len(Keys)):
        rho[int(Keys[i],2)][int(Keys[i],2)]=Vals[i]/tot
    noisy_value=numpy.real(numpy.sum(numpy.diag(rho@H_3.to_matrix())))
#     with open('error_val2.txt','+a') as f:
#         print(str(noisy_value),file=f)
    print(noisy_value,'\n')
    return noisy_value

In [41]:
H_1=circs_and_rotH[0][1]
H_2=circs_and_rotH[1][1]
H_3=circs_and_rotH[2]
circ=var_form_base.construct_circuit(parameters=params)
circ_arr=[circ.copy(),circ.copy(),circ.copy()]
circ1=circ_arr[0]+circs_and_rotH[0][0]
circ2=circ_arr[1]+circs_and_rotH[1][0]
circ3=circ_arr[2]

In [42]:
circ=qiskit.transpile(circ,basis_gates=["u1", "u2", "u3", "cx"],optimization_level=3)
circ1=qiskit.transpile(circ1,basis_gates=["u1", "u2", "u3", "cx"],optimization_level=3)
circ2=qiskit.transpile(circ2,basis_gates=["u1", "u2", "u3", "cx"],optimization_level=3)
circ3=qiskit.transpile(circ3,basis_gates=["u1", "u2", "u3", "cx"],optimization_level=3)

In [43]:
import json

In [44]:
fileObject = open("H1.json", "r")
jsonContent = fileObject.read()
H_1 = json.loads(jsonContent)
fileObject = open("H2.json", "r")
jsonContent = fileObject.read()
H_2 = json.loads(jsonContent)
fileObject = open("H3.json", "r")
jsonContent = fileObject.read()
H_3 = json.loads(jsonContent)
H_1=PauliSumOp.from_list(H_1)
H_2=PauliSumOp.from_list(H_2)
H_3=PauliSumOp.from_list(H_3)

In [45]:
state1=numpy.array(execute(circ1,Aer.get_backend('statevector_simulator')).result().get_statevector())
ideal_value1=numpy.real(numpy.conjugate(state1)@H_1.to_matrix()@state1)
state2=numpy.array(execute(circ2,Aer.get_backend('statevector_simulator')).result().get_statevector())
ideal_value2=numpy.real(numpy.conjugate(state2)@H_2.to_matrix()@state2)
state3=numpy.array(execute(circ3,Aer.get_backend('statevector_simulator')).result().get_statevector())
ideal_value3=numpy.real(numpy.conjugate(state3)@H_3.to_matrix()@state3)
ideal_value1+ideal_value2+ideal_value3

-19.184209626793674

In [46]:
initial_layout1=[1,0,3,4,5,6]
initial_layout2=[1,0,3,4,5,6]
initial_layout3=[1,0,3,4,5,6]

In [47]:
# f=open('circuit1.txt','w')
# f.close()
# f=open('circuit2.txt','w')
# f.close()
# f=open('circuit3.txt','w')
# f.close()
# f=open("circuit1_counts_not_filtered.json", "w")
# f.close()
# f=open("circuit2_counts_not_filtered.json", "w")
# f.close()
# f=open("circuit3_counts_not_filtered.json", "w")
# f.close()
# f=open("circuit1_counts_filtered.json", "w")
# f.close()
# f=open("circuit2_counts_filtered.json", "w")
# f.close()
# f=open("circuit3_counts_filtered.json", "w")
# f.close()
noisy_value1 = executor1(circ1,useSubspaceFilter=True,measure_mitiq=True)
noisy_value2 = executor2(circ2,useSubspaceFilter=True,measure_mitiq=True)
noisy_value3 = executor3(circ3,useSubspaceFilter=True,measure_mitiq=True)

Executing circuit with (974, 349) gates using 16384 shots.
-0.19230435949713584 

Executing circuit with (755, 317) gates using 16384 shots.
0.0044409266385227325 

Executing circuit with (767, 329) gates using 16384 shots.
-14.699210776660205 



In [39]:
from mitiq import zne
from mitiq.interface import convert_to_mitiq, convert_from_mitiq

In [48]:
factory1 = zne.inference.RichardsonFactory(scale_factors=[1,2,3])#zne.inference.AdaExpFactory(steps=3, asymptote=0.001)
factory2 = zne.inference.RichardsonFactory(scale_factors=[1,2,3])#zne.inference.AdaExpFactory(steps=5, asymptote=0.001)
factory3 = zne.inference.RichardsonFactory(scale_factors=[1,2,3])

In [49]:
zne_value1 = zne.execute_with_zne(circ1, executor1, factory=factory1)
print('\n')
zne_value2 = zne.execute_with_zne(circ2, executor2, factory=factory2)
print('\n')
zne_value3 = zne.execute_with_zne(circ3, executor3, factory=factory3)

Executing circuit with (1143, 382) gates using 16384 shots.
-0.2149114537680945 

Executing circuit with (2300, 639) gates using 16384 shots.
-0.2186666649571678 

Executing circuit with (3230, 657) gates using 16384 shots.
-0.1898168533372482 



Executing circuit with (892, 326) gates using 16384 shots.
-0.03335727307624465 

Executing circuit with (1755, 505) gates using 16384 shots.
-0.037905712358999304 

Executing circuit with (2513, 603) gates using 16384 shots.
-0.004413785393222985 



Executing circuit with (919, 353) gates using 16384 shots.
-15.699650754200103 

Executing circuit with (1716, 490) gates using 16384 shots.
-15.806074316394415 

Executing circuit with (2519, 609) gates using 16384 shots.
-15.565928122957441 



In [50]:
unmitigated_error1 = abs(ideal_value1 - noisy_value1)
zne_error1 = abs(ideal_value1 - zne_value1)
print(f"1st term Estimation error without Mitiq   : {unmitigated_error1}")
print(f"1st term Estimation error with Mitiq (ZNE): {zne_error1}")
unmitigated_error2 = abs(ideal_value2 - noisy_value2)
zne_error2 = abs(ideal_value2 - zne_value2)
print(f"2nd term Estimation error without Mitiq   : {unmitigated_error2}")
print(f"2nd term Estimation error with Mitiq (ZNE): {zne_error2}")
unmitigated_error3 = abs(ideal_value3 - noisy_value3)
zne_error3 = abs(ideal_value3 - zne_value3)
print(f"3rd term Estimation error without Mitiq   : {unmitigated_error3}")
print(f"3rd term Estimation error with Mitiq (ZNE): {zne_error3}")

1st term Estimation error without Mitiq   : 0.648013887731606
1st term Estimation error with Mitiq (ZNE): 0.6617670274587139
2nd term Estimation error without Mitiq   : 0.133298361061288
2nd term Estimation error with Mitiq (ZNE): 0.13808896687780647
3rd term Estimation error without Mitiq   : 3.515823168481962
3rd term Estimation error with Mitiq (ZNE): 2.9683765087676726


In [51]:
print("exact energy", ideal_value1+ideal_value2+ideal_value3)
print("estimated energy without Mitiq", noisy_value1+noisy_value2+noisy_value3)
print("estimated energy with Mitiq", zne_value1+zne_value2+zne_value3)

exact energy -19.184209626793674
estimated energy without Mitiq -14.887074209518818
estimated energy with Mitiq -15.415977123689482


In [52]:
print(f"ZNE reduced the estimation error by {(unmitigated_error1 - zne_error1) / unmitigated_error1 :.1%}.")
print(f"ZNE reduced the estimation error by {(unmitigated_error2 - zne_error2) / unmitigated_error2 :.1%}.")
print(f"ZNE reduced the estimation error by {(unmitigated_error3 - zne_error3) / unmitigated_error3 :.1%}.")

ZNE reduced the estimation error by -2.1%.
ZNE reduced the estimation error by -3.6%.
ZNE reduced the estimation error by 15.6%.
